In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import optuna
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import MeanSquaredError

In [ ]:
# Optuna optimization

df = pd.read_excel('Data-files/homonuclear-159-15features-degree2-pairwise.xlsx') 
feature = df.iloc[:, 2:]  
target = df.iloc[:, 1].values  
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=16)
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

def create_model(params):
    units_1 = params['units_1']
    units_2 = params['units_2']
    units_3 = params['units_3']
    dropout_rate = params['dropout_rate']
    l2_strength = params['l2_strength']
    learning_rate = params['learning_rate']
    
    model = Sequential()
    model.add(Input(shape=(X_train_scaled.shape[1],)))
    model.add(Dense(units_1, activation='relu', kernel_regularizer=l2(l2_strength)))  
    model.add(Dropout(dropout_rate)) 
    model.add(Dense(units_2, activation='relu', kernel_regularizer=l2(l2_strength))) 
    model.add(Dropout(dropout_rate))  
    model.add(Dense(units_3, activation='relu', kernel_regularizer=l2(l2_strength)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))  
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

# Define the objective function
def objective(trial):
    units_1 = trial.suggest_int('units_1', 64, 512, step=32)
    units_2 = trial.suggest_int('units_2', 64, 512, step=32)
    units_3 = trial.suggest_int('units_3', 64, 512, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.05, 0.4) 
    l2_strength = trial.suggest_float('l2_strength', 1e-6, 1e-2, log=True) 
    learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-2, log=True) 

    model = create_model({
        'units_1': units_1,
        'units_2': units_2,
        'units_3': units_3,
        'dropout_rate': dropout_rate,
        'l2_strength': l2_strength,
        'learning_rate': learning_rate
    })

    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=20, min_lr=1e-6)
    history = model.fit(X_train_scaled, y_train, epochs=1000, batch_size=128, validation_data=(X_test_scaled, y_test), 
                        callbacks=[early_stopping, lr_scheduler], verbose=0)
    y_pred = model.predict(X_test_scaled)
    r2 = r2_score(y_test, y_pred)  
    return -r2

# minimize -r2
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)  
print(f"Best trial: {study.best_trial.params}")
print(f"Best R2: {-study.best_trial.value}")

In [ ]:
# train DNN with best parameters
df = pd.read_excel('Data-files/homonuclear-159-15features-degree2-pairwise.xlsx')
feature = df.iloc[:, 2:]
target = df.iloc[:, 1].values  

X_train, X_test, y_train, y_test= train_test_split(feature, target, test_size=0.2, random_state=16)
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)
joblib.dump(ss, 'trained-DNN/scaler_model.pkl') 

model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(416, activation='relu', kernel_regularizer=l2(0.00014548420227266957)))  
model.add(Dropout(0.13609925977494042))  
model.add(Dense(384, activation='relu', kernel_regularizer=l2(0.00014548420227266957)))
model.add(Dropout(0.13609925977494042))  
model.add(Dense(352, activation='relu', kernel_regularizer=l2(0.00014548420227266957))) #0.00014548420227266957
model.add(Dropout(0.13609925977494042))  #0.13609925977494042
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate= 6.673095776932314e-05), loss='mse') 

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=20, min_lr=1e-6)
history = model.fit(X_train_scaled, y_train, epochs=1000, batch_size=64, validation_data=(X_test_scaled, y_test), 
                    callbacks=[early_stopping, lr_scheduler], verbose=1)

model.save('trained-DNN/trained_model.keras')

# predict
y_train_pred = model.predict(X_train_scaled).flatten()
y_test_pred = model.predict(X_test_scaled).flatten()
train_r, _ = pearsonr(y_train, y_train_pred)
test_r, _ = pearsonr(y_test, y_test_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

train_label = f"Training: r/{train_r:.2f}, RMSE/{train_rmse:.2f}"
test_label = f"Test: r/{test_r:.2f}, RMSE/{test_rmse:.2f}"

plt.figure(figsize=(6, 6))
plt.plot([min(np.min(y_train), np.min(y_test)), max(np.max(y_train), np.max(y_test))], 
         [min(np.min(y_train), np.min(y_test)), max(np.max(y_train), np.max(y_test))], color='black', linestyle='--', linewidth=1)
plt.scatter(y_train, y_train_pred, color='#8DB8F1', label = train_label)
plt.scatter(y_test, y_test_pred, color='#F47575', label = test_label)
plt.xlabel(r'Experimental Δlg$(k_1)$', fontsize=16)
plt.ylabel(r'Predicted Δlg$(k_1)$', fontsize=16)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=15)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.ensemble import  GradientBoostingRegressor

gbr_model = GradientBoostingRegressor(n_estimators=1500, learning_rate = 0.1)
gbr_model.fit(X_train_scaled, y_train)

# Predict
y_pred_train = gbr_model.predict(X_train_scaled)
y_pred_test = gbr_model.predict(X_test_scaled)
train_r, _ = pearsonr(y_train, y_pred_train)
test_r, _ = pearsonr(y_test, y_pred_test)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f"Gradient Boosting Regressor:")
print(f"Train_r: {train_r:.2f}, Test_r: {test_r:.2f}")
print(f"Train RMSE: {train_rmse:.2f}, Test RMSE: {test_rmse:.2f}")

train_label = f"Training: r/{train_r:.2f}, RMSE/{train_rmse:.2f}"
test_label = f"Test: r/{test_r:.2f}, RMSE/{test_rmse:.2f}"

# Plot results
plt.figure(figsize=(7, 7))
plt.plot([min(np.min(y_train), np.min(y_test)), max(np.max(y_train), np.max(y_test))], 
         [min(np.min(y_train), np.min(y_test)), max(np.max(y_train), np.max(y_test))], color='black', linestyle='--', linewidth=1)
plt.scatter(y_train, y_pred_train, color='#8DB8F1', label=train_label, s=30)
plt.scatter(y_test, y_pred_test, color='#F47575', label=test_label, s=30)
plt.xlabel(r'Experimental Δlg$(k_1)$', fontsize=16)
plt.ylabel(r'Predicted Δlg$(k_1)$', fontsize=16)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=15)
plt.show() 

In [ ]:
from sklearn.linear_model import LinearRegression

mlr_model = LinearRegression()
mlr_model.fit(X_train_scaled, y_train)

# Predict
y_pred_train = mlr_model.predict(X_train_scaled)
y_pred_test = mlr_model.predict(X_test_scaled)
train_r, _ = pearsonr(y_train, y_pred_train)
test_r, _ = pearsonr(y_test, y_pred_test)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f"XGBRegressor:")
print(f"Train_r: {train_r:.2f}, Test_r: {test_r:.2f}")
print(f"Train RMSE: {train_rmse:.2f}, Test RMSE: {test_rmse:.2f}")

train_label = f"Training: r/{train_r:.2f}, RMSE/{train_rmse:.2f}"
test_label = f"Test: r/{test_r:.2f}, RMSE/{test_rmse:.2f}"

# Plot results
plt.figure(figsize=(7, 7))
plt.plot([min(np.min(y_train), np.min(y_test)), max(np.max(y_train), np.max(y_test))], 
         [min(np.min(y_train), np.min(y_test)), max(np.max(y_train), np.max(y_test))], color='black', linestyle='--', linewidth=1)
plt.scatter(y_train, y_pred_train, color='#8DB8F1', label=train_label, s=30)
plt.scatter(y_test, y_pred_test, color='#F47575', label=test_label, s=30)
plt.xlabel(r'Experimental Δlg$(k_1)$', fontsize=16)
plt.ylabel(r'Predicted Δlg$(k_1)$', fontsize=16)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=15)
plt.show() 

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

param_grid = {
    'kernel': ['rbf'], #, 'linear'
    'C': [1.0, 5.0],
    'gamma': ['scale', 'auto'],
    'epsilon': [0.05, 0.5]
}
svr_model = SVR() 
grid_search = GridSearchCV(estimator=svr_model, param_grid=param_grid, cv=10, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)
best_svr_model = grid_search.best_estimator_
print("Best hyperparameters:", grid_search.best_params_)

# Predict
y_pred_train = best_svr_model.predict(X_train_scaled)
y_pred_test = best_svr_model.predict(X_test_scaled)
train_r, _ = pearsonr(y_train, y_pred_train)
test_r, _ = pearsonr(y_test, y_pred_test)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f"XGBRegressor:")
print(f"Train_r: {train_r:.2f}, Test_r: {test_r:.2f}")
print(f"Train RMSE: {train_rmse:.2f}, Test RMSE: {test_rmse:.2f}")

train_label = f"Training: r/{train_r:.2f}, RMSE/{train_rmse:.2f}"
test_label = f"Test: r/{test_r:.2f}, RMSE/{test_rmse:.2f}"

# Plot results
plt.figure(figsize=(7, 7))
plt.plot([min(np.min(y_train), np.min(y_test)), max(np.max(y_train), np.max(y_test))], 
         [min(np.min(y_train), np.min(y_test)), max(np.max(y_train), np.max(y_test))], color='black', linestyle='--', linewidth=1)
plt.scatter(y_train, y_pred_train, color='#8DB8F1', label=train_label, s=30)
plt.scatter(y_test, y_pred_test, color='#F47575', label=test_label, s=30)
plt.xlabel(r'Experimental Δlg$(k_1)$', fontsize=16)
plt.ylabel(r'Predicted Δlg$(k_1)$', fontsize=16)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=15)
plt.show() 